In [1]:
from winners_only import *
from log import *
from historical import *

import pandas as pd
import numpy as np
import math

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv("data/updated_data.csv")

In [3]:
X_train, y_train, X_test, y_test = historical_cleaning(df)

In [4]:
X_train.head()

,Fighter,no_of_rounds,avg_BODY_att,avg_BODY_landed,avg_CLINCH_att,avg_CLINCH_landed,avg_DISTANCE_att,avg_DISTANCE_landed,avg_GROUND_att,avg_GROUND_landed,avg_HEAD_att,avg_HEAD_landed,avg_KD,avg_LEG_att,avg_LEG_landed,avg_PASS,avg_REV,avg_SIG_STR_att,avg_SIG_STR_landed,avg_SIG_STR_pct,avg_SUB_ATT,avg_TD_att,avg_TD_landed,avg_TD_pct,avg_TOTAL_STR_att,avg_TOTAL_STR_landed,avg_opp_BODY_att,avg_opp_BODY_landed,avg_opp_CLINCH_att,avg_opp_CLINCH_landed,avg_opp_DISTANCE_att,avg_opp_DISTANCE_landed,avg_opp_GROUND_att,avg_opp_GROUND_landed,avg_opp_HEAD_att,avg_opp_HEAD_landed,avg_opp_KD,avg_opp_LEG_att,avg_opp_LEG_landed,avg_opp_PASS,avg_opp_REV,avg_opp_SIG_STR_att,avg_opp_SIG_STR_landed,avg_opp_SIG_STR_pct,avg_opp_SUB_ATT,avg_opp_TD_att,avg_opp_TD_landed,avg_opp_TD_pct,avg_opp_TOTAL_STR_att,avg_opp_TOTAL_STR_landed,total_rounds_fought,total_time_fought(seconds),total_title_bouts,Height_cms,Reach_cms,Weight_lbs,age,Year,Win_Pct
116,Jon Jones,5,19.722222,14.611111,16.111111,11.277778,78.333333,38.500000,16.722222,12.222222,70.055556,31.888889,0.277778,21.388889,15.500000,1.444444,0.000000,111.166667,62.000000,0.573889,0.555556,4.000000,2.000000,0.576667,126.555556,76.000000,14.222222,7.777778,14.444444,7.444444,71.111111,22.111111,0.611111,0.333333,62.888889,15.722222,0.000000,9.055556,6.388889,0.000000,0.0,86.166667,29.888889,0.331111,0.055556,2.222222,0.111111,0.013333,95.277778,38.055556,55.0,844.055556,11.0,193.04,213.36,205.0,31.0,2018,0.888889
117,Nathaniel Wood,3,7.000000,3.000000,0.000000,0.000000,97.000000,33.000000,0.000000,0.000000,73.000000,20.000000,0.000000,17.000000,10.000000,1.000000,0.000000,97.000000,33.000000,0.340000,1.000000,0.000000,0.000000,0.000000,97.000000,33.000000,8.000000,3.000000,2.000000,2.000000,86.000000,35.000000,0.000000,0.000000,72.000000,27.000000,0.000000,8.000000,7.000000,0.000000,0.0,88.000000,37.000000,0.420000,0.000000,1.000000,0.000000,0.000000,88.000000,37.000000,2.0,438.000000,0.0,167.64,175.26,135.0,25.0,2018,1.000000
118,Uriah Hall,3,9.000000,5.769231,3.230769,2.692308,47.153846,22.461538,6.769231,4.692308,39.461538,16.307692,0.307692,8.692308,7.769231,0.076923,0.307692,57.153846,29.846154,0.535385,0.153846,1.384615,0.538462,0.249231,65.307692,37.230769,9.615385,6.384615,8.230769,6.000000,49.000000,20.384615,8.230769,5.000000,44.000000,14.923077,0.230769,11.846154,10.076923,1.923077,0.0,65.461538,31.384615,0.502308,0.230769,2.692308,0.769231,0.263077,86.384615,49.384615,26.0,521.153846,1.0,182.88,200.66,185.0,34.0,2018,0.461538
119,Curtis Millender,3,12.500000,8.500000,3.000000,1.500000,93.000000,38.500000,4.000000,2.500000,78.000000,27.000000,1.000000,9.500000,7.000000,0.000000,0.000000,100.000000,42.500000,0.420000,0.000000,0.500000,0.500000,0.500000,101.000000,43.500000,8.500000,5.000000,1.500000,0.500000,95.500000,33.500000,5.500000,4.500000,69.000000,15.500000,0.000000,25.000000,18.000000,1.000000,0.0,102.500000,38.500000,0.375000,0.000000,1.500000,1.000000,0.750000,113.500000,49.000000,5.0,728.500000,0.0,187.96,193.04,170.0,31.0,2018,1.000000
120,Edson Barboza,3,19.052632,11.789474,1.421053,0.947368,96.210526,38.842105,2.315789,1.421053,64.684211,16.894737,0.473684,16.210526,12.526316,0.105263,0.000000,99.947368,41.210526,0.443158,0.052632,0.736842,0.368421,0.192632,101.947368,42.842105,9.736842,5.421053,1.421053,0.789474,97.631579,33.421053,15.631579,11.157895,97.421053,34.052632,0.105263,7.526316,5.894737,1.105263,0.0,114.684211,45.368421,0.397368,0.263158,3.947368,0.789474,0.219474,130.526316,59.263158,48.0,680.052632,0.0,180.34,190.50,155.0,32.0,2018,0.684211


In [12]:
my_list = list(X_train.groupby("Fighter")["no_of_rounds"].count())

In [14]:
freq = {} 
for item in my_list: 
    if (item in freq): 
        freq[item] += 1
    else: 
        freq[item] = 1

In [15]:
freq

{1: 352,
 8: 59,
 2: 217,
 10: 45,
 4: 109,
 3: 181,
 6: 98,
 5: 98,
 11: 31,
 14: 27,
 7: 55,
 18: 11,
 9: 51,
 13: 28,
 21: 6,
 27: 4,
 25: 5,
 16: 9,
 15: 10,
 19: 12,
 17: 13,
 12: 26,
 22: 4,
 20: 11,
 28: 3,
 26: 2,
 23: 5,
 30: 1,
 24: 4}

In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7989 entries, 116 to 10283
Data columns (total 59 columns):
Fighter                       7989 non-null object
no_of_rounds                  7989 non-null int64
avg_BODY_att                  7989 non-null float64
avg_BODY_landed               7989 non-null float64
avg_CLINCH_att                7989 non-null float64
avg_CLINCH_landed             7989 non-null float64
avg_DISTANCE_att              7989 non-null float64
avg_DISTANCE_landed           7989 non-null float64
avg_GROUND_att                7989 non-null float64
avg_GROUND_landed             7989 non-null float64
avg_HEAD_att                  7989 non-null float64
avg_HEAD_landed               7989 non-null float64
avg_KD                        7989 non-null float64
avg_LEG_att                   7989 non-null float64
avg_LEG_landed                7989 non-null float64
avg_PASS                      7989 non-null float64
avg_REV                       7989 non-null float64
avg_SIG_S